In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
===================================================================

Simulation of search on a NK landscape

You have 100 periods and can try one choice configuration
in each period. You will then be told the performance of this choice
configuration.

Your score is the maximum performance
of the choice configurations you tried during the 100 periods

You can change the search algorithm - look for a segment starting with

=======================
SEARCH ALGORITHM
=======================

That segment specifies the choice configuration tested in any
given period, t. The choice can depend on the period (t), on n,k,
and also on the choices and performance observed so far.
Your goal is to find a search strategy that maximizes the average
final performance (which will be based on, at least, 100 000 simulations)

You can change that segment in any way
But the choice can only depend on t,n,k,choices and performances
t = period
choices = choices vectors so far:
choice[0] = choice vector in period 1, choice[1] = choice vector in period 2 etc
performances = performances so far
performances [0] = performance in period 1, perfs[1] = performance in period 2 etc

===================================================================
"""

import numpy as np


# Fixed parameters
n = 10 # N in NK
k = 3 # K in NK
number_of_periods  = 100 # Number of periods (keep this at 100)
number_of_simulations = 100000 # Number of simulations/iterations (you can vary this at your will)


"""
===================================================================
Here we define the performance function and the dependencies
"""

class PerfFuncCallError(Exception):
    pass


def perffunc(ac, k, n, dep, fitn, call_counter, t):
    # Check if perffunc has been called more than once in this period
    if call_counter[t - 1] >= 1:
        raise PerfFuncCallError(f"Performance function was called more than once in period {t}.")

    call_counter[t - 1] += 1  # Increment the call counter for the current period

    ll = np.arange(k + 1)
    binv = 2 ** ll
    # binv = 2^0,2^1,...,2^k
    # this is used to index the different choice vectors
    # each choice vector has a number associated with it
    # using the binary representation

    # Next
    # create a matrix "mat" with n rows and k+1 columns
    # The matrix specifies which dimensions the fitness contribution
    # of a given dimension depends on

    mat = np.zeros((n, k + 1), dtype=int)
    mat[:, 0] = ac[:n]   # the fitness contribution
    mat[:, 1:] = ac[dep[:n, :]]
    # Compute binary numbers for each row using matrix multiplication
    nos = np.dot(mat, binv)
    # this gives each choice vector in the rows a unique number
    # for example 0101 = 5 since
    # (starting from the right): 1*2^0 + 0*2^1 + 1*2^2 + 0*2^3 = 1+0+4+0 = 5
    # then I know that the fitness contribution of 0101 for row i
    # (corresponding to dimension i) is fitn(i,5)

    # Compute performance: just find relevant value in fitn matrix
    perf = np.mean(fitn[np.arange(n), nos])

    return perf


def depend(n,k):
    '''
    Here select interdependencies: which k others does a given dimension depend on?
    1) np.arange(n) = list of integers 0,1,2,..,n-1
    2) np.delete(np.arange(n), i): we delete the number i (because dimension i should not be selected)
    3) np.random.choice: now we select k values from the resulting list
    '''

    dep = np.zeros((n, k), dtype=int)
    for i in range(n):
        dep[i] = np.random.choice(np.delete(np.arange(n), i), k, replace=False)

    return dep

"""
End of functions
===================================================================
"""


 # repeat the simulation for many runs
scores = np.zeros(number_of_simulations, dtype=float) # array to store final scores (max performance of a single simulation)


for ns in range(0, number_of_simulations): # repeated over the number of simulations

    choices = np.zeros((number_of_periods,n), dtype=int)
    performances = np.zeros(number_of_periods, dtype=float)
    call_counter = np.zeros(number_of_periods, dtype=int)  # Initialize the call counter for each period

    # Here select fitness values and dependencies
    fitness_values = np.random.rand(n, 2**(k+1))
    dependencies = depend(n, k)

    # loop from period 1 to period "per"
    for t in range(1, number_of_periods):

        ''' ===================================================================
        THE SEARCH ALGORITHM: This is the only part you can edit
        ===================================================================='''

        # fixed search parameters
        unique_check = 0
        difference_while = 0 
        difference_while_exitcondition = 0 

        # tuned search parameters
        numrandom_initial_testsA = 17 # technically any increase in this variable should improve fitness
        difference_thresholdA = 7
        pick_from_top0 = 6 
        extra_mutation_chances = 50 # technically any increase in this variable should improve fitness

        
        choice_configuration_matrixB = choices[:t, :]
        # Initial round random choices and performance
        if t==1:
            choice_configuration = np.random.randint(0, 2, n)
            choices[0] = choice_configuration
            t = 1  # Set t to 1 for the first period
            choice_configuration_matrixB = choices[:t, :]
        elif t<numrandom_initial_testsA:
            while difference_while == 0:
                choice_configuration = np.random.randint(0, 2, n)
                difference_while_exitcondition += 1
                if np.max(np.sum(choice_configuration_matrixB[:] == choice_configuration, axis=1)) < difference_thresholdA:
                    difference_while = 1
                elif difference_while_exitcondition > 10*1024: # sorry I know this exit condition is irresponsible ¯\_(ツ)_/¯. It's been a long year
                    # when this exit condition is met, the algorithm gives up and does a local jump
                    if pick_from_top0 < t:
                        pick_from_top = pick_from_top0
                    else:
                        pick_from_top = t
                    ch0 = np.random.randint(0, pick_from_top)
                    ind = np.argpartition(performances, -pick_from_top)[-pick_from_top:] # the top m performing configurations where m = pick_from_top
                    betteridx = ind[ch0]
                    betterch = choices[betteridx,:] # loosely resembling Jingyi Wu's "Better than Best: Epistemic Landscapes and Diversity of Practice in Science"
    
                    # Then, change the choice on one randomly chosen dimension
                    ch = np.random.randint(0, n) # a random dimension (a random integer between 0 and n-1)
                    choice_configuration = betterch.copy()  # copy the better choice vector
                    choice_configuration[ch] = 1 - betterch[ch]  # Then flip the choice on dimension ch
                    for retry in range(0, extra_mutation_chances): # retry if the new choice has already been tried
                        if any((choice_configuration_matrixB[:]==choice_configuration).all(1)):
                            ch0 = np.random.randint(0, pick_from_top)
                            betteridx = ind[ch0]
                            betterch = choices[betteridx,:] 
                            ch = np.random.randint(0, n) 
                            choice_configuration = betterch.copy()  
                            choice_configuration[ch] = 1 - betterch[ch]
                    difference_while = 1
        else:
            # local searches almost always
            if pick_from_top0 < t:
                pick_from_top = pick_from_top0
            else:
                pick_from_top = t
            ch0 = np.random.randint(0, pick_from_top)
            ind = np.argpartition(performances, -pick_from_top)[-pick_from_top:]
            betteridx = ind[ch0]
            betterch = choices[betteridx,:] # loosely resembling Jingyi Wu's "Better than Best: Epistemic Landscapes and Diversity of Practice in Science"

            # Then, change the choice on one randomly chosen dimension
            ch = np.random.randint(0, n) # a random dimension (a random integer between 0 and n-1)
            choice_configuration = betterch.copy()  # copy the better choice vector
            choice_configuration[ch] = 1 - betterch[ch]  # Then flip the choice on dimension ch
            for retry in range(0, extra_mutation_chances): # retry if the new choice has already been tried
                if any((choice_configuration_matrixB[:]==choice_configuration).all(1)):
                    ch0 = np.random.randint(0, pick_from_top)
                    betteridx = ind[ch0]
                    betterch = choices[betteridx,:] 
                    ch = np.random.randint(0, n) 
                    choice_configuration = betterch.copy()  
                    choice_configuration[ch] = 1 - betterch[ch]  

                # at the end you should have a choice_configuration variable that represents the new location
        while unique_check == 0: #at the very least you can improve the algorithm by insuring that you dont investigate duplicates
            if any((choice_configuration_matrixB[:]==choice_configuration).all(1)):
                choice_configuration = np.random.randint(0, 2, n)
            else:
                unique_check = 1

        ''' ===================================================================
        THE SEARCH ALGORITHM ENDS HERE
        ===================================================================='''

                        # we calculate the performance of this choice vector
        current_performance=perffunc(choice_configuration, k, n, dependencies, fitness_values, call_counter, t)


        # we save performance and choices to the respective vectors
        choices[t-1] = choice_configuration  # this function records your current choice
        performances[t-1] = current_performance   # and performance associated with it

    # after all time periods (t) we calculate the score
    scores[ns] = np.max(performances)   # The score is the max of this run

avg_score = np.mean(scores) # compute the average
print("Average score is: %.4f" % avg_score)



Average score is: 0.7523


In [2]:
print("Average score is: %.6f" % avg_score)

Average score is: 0.752296
